# Convert original Hologram output format (numpy record array) into other dataFormat

- author Sylvie Dagoret-Campagne
- creation date 2024-09-23
- last update : 2024-09-25, version v3
- affiliation : IJCLab
- Kernel @usdf **w_2024_16**
- Office emac : mamba_py311
- Home emac : base (conda)
- laptop : conda_py310


In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'


# new color correction model
import pickle


from astropy.table import Table
from astropy.io import fits

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

### Load Holo fit results

In [ ]:
version_results = "v3"

In [ ]:
if version_results == "":
    atmfilename = "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy"
elif version_results == "v2":
    atmfilename = "auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_PeekFinder.npy"
elif version_results == "v3":
    atmfilename = "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z.npy"

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)
df_spec

In [ ]:
#subtract the a bug number to have normal dates
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)
df_spec["nightObs"]

## Convert into files

In [ ]:
flag_HDF5 = True
flag_PARQUET = True
flag_FITS = True
flag_SQL = True

In [ ]:
output_fn_root = re.findall("(.*)[.]npy$",atmfilename)
if len(output_fn_root)>0:
    output_fn_root = output_fn_root[0]
else:
    print("error in extracting root for filename {atmfilename}, rootfilename = ",output_fn_root)
    

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

if flag_SQL:
    output_fn = f"{output_fn_root}.sql"
    try:
        with engine.begin() as connection:
            df_spec.to_sql(output_fn,con=connection,if_exists='replace')   
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)   
    finally:
        if os.path.exists(output_fn):
            print(f" file {output_fn} created")
        else:
            print(f" >>>>> file {output_fn} NOT created")

In [ ]:
if flag_HDF5:
    output_fn = f"{output_fn_root}.hdf5"
    try:
        df_spec.to_hdf(output_fn,key='data', format='table', data_columns=True)   
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)   
    finally:
        if os.path.exists(output_fn):
            print(f" file {output_fn} created")
        else:
            print(f" >>>>> file {output_fn} NOT created")

In [ ]:
# not working with pyarraow then try fastparquet
#! pip install fastparquet

#import pyarrow.dataset as ds
#parquet_format = ds.ParquetFileFormat()
#file_options = parquet_format.make_write_options(coerce_timestamps='us', allow_truncated_timestamps=True)


if flag_PARQUET:
    output_fn = f"{output_fn_root}.parquet.gzip"
    #output_fn = f"{output_fn_root}.parquet"
    try:
        df_spec.to_parquet(output_fn,compression='gzip',engine='fastparquet')
        #ds.write_dataset(df_spec, file_options=file_options,base_dir='./')
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)   
    finally:
        if os.path.exists(output_fn):
            print(f" file {output_fn} created")
        else:
            print(f" >>>>> file {output_fn} NOT created")

In [ ]:
if flag_FITS:
    output_fn = f"{output_fn_root}.fits"
    t = Table.from_pandas(df_spec)
    try:
        t.write(output_fn,format="fits",overwrite=True)
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)   
    finally:
        if os.path.exists(output_fn):
            print(f" file {output_fn} created")
        else:
            print(f" >>>>> file {output_fn} NOT created")